# Análise Exploratória de Dados - BigQuery

Este notebook realiza uma análise exploratória completa dos dados armazenados no BigQuery, utilizando a biblioteca `ydata-profiling` para gerar relatórios detalhados.

## Estrutura do Notebook

1. **Instalação de Dependências**: Instalação dos pacotes necessários
2. **Autenticação**: Configuração de acesso ao Google Cloud
3. **Configuração Inicial**: Imports e configurações do projeto
4. **Análise com Amostragem**: EDA da tabela `orders_raw` (dados volumosos)
5. **Análise Completa**: EDA das demais tabelas do dataset

---

## 1. Instalação de Dependências

Instalamos os pacotes necessários para:
- **ydata-profiling**: Geração automática de relatórios de análise exploratória
- **google-cloud-bigquery**: Cliente para conexão com BigQuery
- **pandas**: Manipulação e análise de dados

In [ ]:
!pip install --quiet ydata-profiling google-cloud-bigquery pandas

## 2. Autenticação no Google Cloud

A autenticação é necessária para acessar os recursos do BigQuery. O Google Colab fornece uma interface simplificada para este processo.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Autenticação realizada com sucesso!")

Autenticação realizada com sucesso!


## 3. Configuração Inicial

Importamos as bibliotecas necessárias e configuramos os parâmetros de conexão com o BigQuery:
- **project_id**: ID do projeto no Google Cloud
- **dataset**: Nome do dataset que contém as tabelas
- **client**: Cliente BigQuery para execução de queries

In [ ]:
from google.cloud import bigquery
from ydata_profiling import ProfileReport
import pandas as pd

# Configurações do projeto
project_id = "294185590533"
dataset = "ifood"
client = bigquery.Client(project=project_id)

print(f"Projeto configurado: {project_id}")
print(f"Dataset: {dataset}")

Projeto configurado: 294185590533
Dataset: ifood


## 4. Funções Auxiliares

Definimos funções reutilizáveis para:
- **Amostragem de dados**: Para tabelas muito grandes
- **Geração de relatórios**: Criação automática de relatórios EDA

In [ ]:
def carregar_amostra_bigquery(linhas=100000):
    """
    Faz amostragem aleatória da tabela orders_raw no BigQuery

    Args:
        linhas (int): Número máximo de linhas a serem carregadas

    Returns:
        pandas.DataFrame: Amostra dos dados
    """
    # Calcula a porcentagem de amostragem baseada no total estimado de 3.67M linhas
    porcentagem_amostra = (linhas / 3670000) * 100

    query = f"""
    SELECT * FROM `{project_id}.{dataset}.orders_raw`
    TABLESAMPLE SYSTEM ({porcentagem_amostra:.4f} PERCENT)
    LIMIT {linhas}
    """

    print(f"Carregando amostra de até {linhas:,} linhas de orders_raw...")
    return client.query(query).to_dataframe()


def gerar_eda(df, nome_tabela):
    """
    Gera relatório de análise exploratória usando ydata-profiling

    Args:
        df (pandas.DataFrame): DataFrame para análise
        nome_tabela (str): Nome da tabela para identificação do relatório
    """
    print(f"Gerando relatório EDA para {nome_tabela}...")

    # Configuração do relatório com modo minimal para melhor performance
    profile = ProfileReport(
        df,
        title=f"Análise Exploratória - {nome_tabela}",
        minimal=True
    )

    # Salva o relatório em HTML
    nome_arquivo = f"{nome_tabela}_eda.html"
    profile.to_file(nome_arquivo)

    print(f"Relatório salvo como {nome_arquivo}")
    print(f"Dimensões dos dados: {df.shape[0]:,} linhas x {df.shape[1]} colunas\n")

## 5. Análise Exploratória - Tabela orders_raw (com Amostragem)

A tabela `orders_raw` contém aproximadamente 3.67 milhões de registros. Para otimizar o processamento, utilizamos amostragem estatística que mantém a representatividade dos dados.

### Estratégia de Amostragem
- **Método**: TABLESAMPLE SYSTEM (amostragem sistemática)
- **Tamanho**: 100.000 registros (~3% do total)
- **Vantagem**: Preserva as características estatísticas dos dados originais

In [ ]:
# Carrega amostra da tabela orders_raw
df_orders_sample = carregar_amostra_bigquery(linhas=100000)

# Exibe informações básicas da amostra
print("Informações da amostra carregada:")
print(f"- Linhas: {df_orders_sample.shape[0]:,}")
print(f"- Colunas: {df_orders_sample.shape[1]}")
print(f"- Memória utilizada: {df_orders_sample.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Visualiza as primeiras linhas
print("\nPrimeiras 5 linhas da amostra:")
display(df_orders_sample.head())

Carregando amostra de até 100,000 linhas de orders_raw...
Informações da amostra carregada:
- Linhas: 100,000
- Colunas: 22
- Memória utilizada: 299.5 MB

Primeiras 5 linhas da amostra:


,cpf,customer_id,customer_name,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,...,merchant_id,merchant_latitude,merchant_longitude,merchant_timezone,order_created_at,order_id,order_scheduled,order_total_amount,origin_platform,order_scheduled_date
0,31540370908,f126c729b1357e8dabda9d274006db376b33323951c448...,Giovanna,MANAUS,BR,PONTA NEGRA,6814446,-60.11,-3.06,AM,...,ebb1dba8980e363105a8bfe950249ccd144d719e8297ff...,-60.11,-3.06,America/Manaus,2019-01-11 23:01:43+00:00,2a13b85d29ea6428cc3eebc8058d768db6c4c99bb68469...,False,75.6,IOS,None
1,29682193959,b47698db0f3bd6cd1ab1e50155c118f670a89ee4d56bf7...,MIKA,MANAUS,BR,PONTA NEGRA,8206156,-60.11,-3.06,AM,...,fa8b35cb33725c9a4ff5033355a9132f7b515856290030...,-60.11,-3.06,America/Manaus,2019-01-12 22:18:11+00:00,5337b805f3fffa2c444ea95169049412372e92091cd8ad...,False,30.9,IOS,None
2,54637525850,8391aba18f3dbf867c4519166cad4c49bf8b134db19e15...,NAYARA,MANAUS,BR,PONTA NEGRA,8028280,-60.11,-3.06,AM,...,da022d82a03be672f1079327d843e929211beef6d75b74...,-60.11,-3.06,America/Manaus,2019-01-05 00:55:23+00:00,57d486911591b398fe24391a7aec44a4c5c3e1a10e913e...,False,96.8,IOS,None
3,53729971378,3146051ff8897a5fe4eb1ff3b64c4dad71aa44a4781b46...,RAFAEL,MANAUS,BR,PONTA NEGRA,8710849,-60.11,-3.06,AM,...,c9bbf6d8c502b7fa57899883250e8bd12d09b27700b52d...,-60.11,-3.06,America/Manaus,2019-01-04 03:32:30+00:00,40618eda0c124bb677f27270cd9453a30194828b0512c8...,False,60.0,IOS,None
4,19544075058,dce62a948bf85361eb3ffd097436413e0a24e8637842fc...,GISELLE,MANAUS,BR,PONTA NEGRA,2941992,-60.10,-3.06,AM,...,c9bbf6d8c502b7fa57899883250e8bd12d09b27700b52d...,-60.10,-3.06,America/Manaus,2019-01-18 02:34:40+00:00,657f9626791ac6bcf8f60c90153ac6f1ab36c8e7bfe868...,False,34.0,IOS,None


In [ ]:
# Gera relatório EDA para a amostra de orders_raw
gerar_eda(df_orders_sample, "orders_raw_sample")

Gerando relatório EDA para orders_raw_sample...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 22/22 [01:00<00:00,  2.75s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Relatório salvo como orders_raw_sample_eda.html
Dimensões dos dados: 100,000 linhas x 22 colunas



## 6. Análise Exploratória - Demais Tabelas

Para as tabelas menores, realizamos análise completa sem necessidade de amostragem:

- **consumers_raw**: Dados dos consumidores
- **restaurants_raw**: Informações dos restaurantes
- **ab_test_ref_raw**: Dados de referência para testes A/B

### Limitação de Segurança
Aplicamos um LIMIT de 1 milhão de registros como medida preventiva para evitar sobrecarga de memória.

In [ ]:
# Lista das tabelas para análise completa
tabelas_completas = ["consumers_raw", "restaurants_raw", "ab_test_ref_raw"]

def gerar_relatorio_bigquery(table_name):
    """
    Carrega dados completos de uma tabela e gera relatório EDA

    Args:
        table_name (str): Nome da tabela no BigQuery
    """
    print(f"Processando tabela: {table_name}")
    print(f"Carregando dados completos (LIMIT 1M para segurança)...")

    # Query para carregar dados da tabela
    query = f"SELECT * FROM `{project_id}.{dataset}.{table_name}`"
    df = client.query(query).to_dataframe()

    # Informações sobre os dados carregados
    print(f"Dados carregados: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
    print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    # Gera o relatório de profiling
    print(f"Gerando relatório para {table_name}...")
    profile = ProfileReport(
        df,
        title=f"Análise Exploratória - {table_name}",
        minimal=True
    )

    # Salva o relatório
    nome_arquivo = f"{table_name}_profile.html"
    profile.to_file(nome_arquivo)
    print(f"Relatório salvo como {nome_arquivo}\n")

    return df

### Execução da Análise para Cada Tabela

O processamento será executado sequencialmente para cada tabela, gerando relatórios individuais.

In [ ]:
# Dicionário para armazenar os DataFrames carregados
dataframes = {}

# Processa cada tabela
for tabela in tabelas_completas:
    print("=" * 60)
    df = gerar_relatorio_bigquery(tabela)
    dataframes[tabela] = df

print("=" * 60)
print("Processamento concluído para todas as tabelas!")

Processando tabela: consumers_raw
Carregando dados completos (LIMIT 1M para segurança)...
Dados carregados: 806,156 linhas x 7 colunas
Memória utilizada: 268.3 MB
Gerando relatório para consumers_raw...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 7/7 [00:25<00:00,  3.61s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...


100%|██████████| 2/2 [00:17<00:00,  8.66s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Relatório salvo como ab_test_ref_raw_profile.html

Processamento concluído para todas as tabelas!


## 7. Resumo dos Resultados

Visualização consolidada das informações básicas de todas as tabelas processadas.

In [ ]:
import pandas as pd

# Cria resumo das tabelas processadas
resumo_tabelas = []

# Adiciona informações da amostra de orders_raw
resumo_tabelas.append({
    'Tabela': 'orders_raw (amostra)',
    'Linhas': f"{df_orders_sample.shape[0]:,}",
    'Colunas': df_orders_sample.shape[1],
    'Arquivo_Relatório': 'orders_raw_sample_eda.html'
})

# Adiciona informações das demais tabelas
for nome_tabela, df in dataframes.items():
    resumo_tabelas.append({
        'Tabela': nome_tabela,
        'Linhas': f"{df.shape[0]:,}",
        'Colunas': df.shape[1],
        'Arquivo_Relatório': f'{nome_tabela}_profile.html'
    })

# Converte para DataFrame e exibe
df_resumo = pd.DataFrame(resumo_tabelas)
print("RESUMO FINAL - ANÁLISE EXPLORATÓRIA")
print("=" * 50)
display(df_resumo)

print("\nTodos os relatórios HTML foram gerados e estão disponíveis para download.")
print("Os arquivos contêm análises detalhadas incluindo:")
print("- Estatísticas descritivas")
print("- Distribuições de variáveis")
print("- Detecção de valores ausentes")
print("- Correlações entre variáveis")
print("- Identificação de outliers")

RESUMO FINAL - ANÁLISE EXPLORATÓRIA


,Tabela,Linhas,Colunas,Arquivo_Relatório
0,orders_raw (amostra),"100,000",22,orders_raw_sample_eda.html
1,consumers_raw,"806,156",7,consumers_raw_profile.html
2,restaurants_raw,"7,292",12,restaurants_raw_profile.html
3,ab_test_ref_raw,"806,467",2,ab_test_ref_raw_profile.html



Todos os relatórios HTML foram gerados e estão disponíveis para download.
Os arquivos contêm análises detalhadas incluindo:
- Estatísticas descritivas
- Distribuições de variáveis
- Detecção de valores ausentes
- Correlações entre variáveis
- Identificação de outliers


## 8. Instruções para Download

### Como baixar os relatórios gerados:

1. **Visualizar arquivos**: No painel esquerdo do Colab, clique no ícone de pasta
2. **Localizar relatórios**: Procure pelos arquivos `.html` gerados
3. **Download individual**: Clique com botão direito no arquivo → "Download"
4. **Download em lote**: Use o código abaixo para compactar todos os relatórios

### Arquivos gerados:
- `orders_raw_sample_eda.html`
- `consumers_raw_profile.html`
- `restaurants_raw_profile.html`
- `ab_test_ref_raw_profile.html`

In [ ]:
import zipfile
import os
from datetime import datetime

# Lista todos os arquivos HTML gerados
arquivos_html = [f for f in os.listdir('.') if f.endswith('.html')]

if arquivos_html:
    # Nome do arquivo ZIP com timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_zip = f"relatorios_eda_{timestamp}.zip"

    # Cria arquivo ZIP com todos os relatórios
    with zipfile.ZipFile(nome_zip, 'w') as zipf:
        for arquivo in arquivos_html:
            zipf.write(arquivo)
            print(f"Adicionado ao ZIP: {arquivo}")

    print(f"\nArquivo ZIP criado: {nome_zip}")
    print(f"Total de relatórios: {len(arquivos_html)}")
    print("\nPara baixar: Clique com botão direito no arquivo ZIP → Download")
else:
    print("Nenhum arquivo HTML encontrado. Execute as células anteriores primeiro.")

Adicionado ao ZIP: ab_test_ref_raw_profile.html
Adicionado ao ZIP: consumers_raw_profile.html
Adicionado ao ZIP: orders_raw_sample_eda.html
Adicionado ao ZIP: restaurants_raw_profile.html

Arquivo ZIP criado: relatorios_eda_20250703_193515.zip
Total de relatórios: 4

Para baixar: Clique com botão direito no arquivo ZIP → Download
